In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
from src.utils import isWorkday, estEndDate, isParentCompleted, isWeekend, calculate_depth, delay
import pyodbc
import random

In [2]:
server = 'LAPTOP-2NSE0JH1\SQLEXPRESS'
database = 'dummy'

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn_str

'DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-2NSE0JH1\\SQLEXPRESS;DATABASE=dummy;Trusted_Connection=yes;'

In [ ]:
try:
    conn = pyodbc.connect(conn_str)

    cursor = conn.cursor()

    task_query = "SELECT * FROM Task"
    tasks = pd.read_sql(task_query, conn)

    project_query = "SELECT * FROM Project"
    projects = pd.read_sql(project_query, conn)

    conn.close()

except pyodbc.Error as e:
    print("Error connecting to SQL Server:", e)

In [3]:
tasks = pd.read_csv('data/task_10.csv')
projects = pd.read_csv('data/project_10.csv')

In [4]:
tasks['ParentTaskID'] = tasks['ParentTaskID'].astype('Int64') 
tasks['StartDate'] = pd.to_datetime(tasks['StartDate'])
tasks['EndDate'] = pd.to_datetime(tasks['EndDate'])
tasks['ActualStartDate'] = pd.to_datetime(tasks['ActualStartDate'])
tasks['ActualEndDate'] = pd.to_datetime(tasks['ActualEndDate'])

In [ ]:
# tasks.to_csv('data/task_10.csv', index=False)
# projects.to_csv('data/project_10.csv', index=False)

In [5]:
curr_date = tasks['StartDate'].min()
task_report = []
project_report = []

while ~tasks['Status'].eq('Completed').all():
    task_today = tasks[(tasks['StartDate'] <= curr_date) & (tasks['Status']!='Completed')]['ID'].tolist()

    for idx in task_today:
        task = tasks.loc[tasks['ID']==idx].iloc[0]
        workday = projects.loc[projects['ID'] == task['ProjectID']].iloc[0]['Workday']
        if isParentCompleted(task, tasks) and isWorkday(curr_date, workday):
            if task['Status'] == 'Not Started':
                task['ActualStartDate'] = str(curr_date)
                task['Status'] = 'On Progress'
                
            if delay(task, tasks, task_today, curr_date):
                task['Progress'] += 0
            else:
                task['Progress'] += 1
                
            if task['Progress'] >= task['Duration']:
                task['ActualEndDate'] = curr_date
                task['Status'] = 'Completed'
            
            if curr_date > task['EndDate'] and task['Status'] == 'On Progress':
                task['Status'] = 'Delayed'
                task['Priority'] = 'Critical'
            
            tasks.loc[tasks['ID']==idx] = task.values
            
            task_report.append({
                'Date': curr_date,
                'ID': task['ID'],
                'Name': task['Name'],
                'StartDate': task['StartDate'],
                'EndDate': task['EndDate'],
                'Cost': task['Cost'],
                'Priority': task['Priority'],
                'Progress': task['Progress'],
                'ProjectID' : task['ProjectID'],
                'Status': task['Status'],
                'Duration': task['Duration'],
                'Trade' : task['Trade'],
                'ActualStartDate': task['ActualStartDate'],
                'ActualEndDate': task['ActualEndDate']
            })
            
    for pid in projects['ID'].tolist():
        project_task = tasks[tasks['ProjectID']==pid]
        if ~project_task['Status'].eq('Not Started').all() and ~project_task['Status'].eq('Completed').all():
            project_report.append({
                'Date' : curr_date,
                'ProjectID' : pid,
                'TotalTask' : len(project_task),
                'StartedTask' : len(project_task[project_task['Status']!='Not Started']),
                'OnGoingTask' : len(project_task[(project_task['Status']!='Not Started') & (project_task['Status']!='Completed')]),
                'DelayedTask' : len(project_task[(project_task['Status']=='Delayed')]),
                'CompletedTask' : len(project_task[(project_task['Status']=='Completed')]),
                'WorkDay' : project_task[(project_task['Status']!='Not Started')]['Progress'].sum(),
                'TotalSpent' : project_task[(project_task['Status']=='Completed')]['Cost'].sum()
            })

    curr_date += timedelta(days=1)

task_reports = pd.DataFrame(task_report)
project_reports = pd.DataFrame(project_report)
task_reports['ActualEndDate'] = task_reports.groupby('ID')['ActualEndDate'].bfill()
task_ns = tasks.copy()
task_ns['ActualStartDate'] = task_ns['StartDate']
task_ns['Date'] = task_ns['ActualStartDate']
task_ns['Priority'] = task_ns.apply(lambda x: 'Critical' if pd.isna(x['ParentTaskID']) else 'Normal', axis=1)
task_ns['Progress'] = 0
task_ns['Status'] = 'Not Started'
task_ns.drop(['ParentTaskID', 'AssigneeID', 'CreateDate'],axis=1, inplace=True)
task_reports = pd.concat([task_reports, task_ns]).reset_index(drop=True)

project_dates = tasks.groupby('ProjectID').agg({'StartDate': 'min', 'EndDate': 'max', 'ActualStartDate':'min','ActualEndDate':'max'}).reset_index()
project_reports = pd.merge(project_reports, project_dates, on='ProjectID', how='left')
project_ns = project_reports.copy()
project_ns = project_reports.groupby('ProjectID').agg({'TotalTask':'max','StartDate':'max','EndDate':'max','ActualStartDate':'max','ActualEndDate':'max'}).reset_index()
project_ns['Date'] = project_ns['ActualStartDate']
project_ns[['StartedTask','OnGoingTask','DelayedTask','CompletedTask', 'WorkDay', 'TotalSpent']] = 0
project_reports = pd.concat([project_reports,project_ns]).reset_index(drop=True)

print(f'Complete all task at {curr_date}')

Complete all task at 2023-09-02 00:00:00


In [ ]:
# task_reports.to_csv('data/task_report_30_4.csv',index=False)
# project_reports.to_csv('data/project_report_30_4.csv',index=False)